In [1]:
import numpy as np

import utilities.constants as const
from core.TrajectoryDF import NumPandasTraj as NumTrajDF
from features.spatial_features import SpatialFeatures as spatial
from features.temporal_features import TemporalFeatures as temporal
from utilities.conversions import Conversions as con
from preprocessing.filters import Filters as filters
import pandas as pd
import time
np.seterr(invalid='ignore')
start = time.time()

In [2]:
%%time

# # Reading the geolife dataset and converting to NumPandasTraj.
# geolife = pd.read_csv('./data/geolife_sample.csv')
# geolife = NumTrajDF(geolife,'lat','lon','datetime','id')
#
# Reading the gulls dataset and converting to NumPandasTraj.
# gulls = pd.read_csv('./data/gulls.csv')
# gulls = NumTrajDF(gulls,
#                  latitude='location-lat',
#                  longitude='location-long',
#                  datetime='timestamp',
#                  traj_id='tag-local-identifier',
#                  rest_of_columns=[])
#
#
# # # Reading the atlantic dataset, cleaning it up and
# # # then converting it to NumPandasTraj.
# atlantic = pd.read_csv('./data/atlantic.csv')
# atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
# def convert_to_datetime(row):
#         this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
#         this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
#         return '{} {}'.format(this_date, this_time)
# atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)
# atlantic = NumTrajDF(atlantic,
#                          latitude='Latitude',
#                          longitude='Longitude',
#                          datetime='DateTime',
#                          traj_id='ID',
#                          rest_of_columns=[])
# atlantic.drop(columns='Time')
# atlantic.drop(columns='Date')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


In [3]:
%%time

pdf = pd.read_csv('./data/dataset_raw_full.csv')
raw_df = NumTrajDF(pdf, 'Latitude', 'Longitude',
                   'Date_Time', 'Id_user')
raw_df.head()


CPU times: user 11.4 s, sys: 760 ms, total: 12.2 s
Wall time: 12.2 s


,,lat,lon,Altitude,Id_perc,Label
DateTime,traj_id,,,,,
2008-03-31 16:00:08,10,41.741415,86.186028,-777.0,20080331160008.plt,taxi
2008-03-31 16:01:07,10,41.737063,86.179470,-777.0,20080331160008.plt,taxi
2008-03-31 16:02:07,10,41.734105,86.172823,-777.0,20080331160008.plt,taxi
2008-03-31 16:03:06,10,41.739110,86.166563,-777.0,20080331160008.plt,taxi
2008-03-31 16:04:05,10,41.744368,86.159987,-777.0,20080331160008.plt,taxi


In [4]:
%%time

# Running all the spatial functions available to check the
# execution time together.

spatial.get_bounding_box(raw_df)
spatial.get_start_location(raw_df)
spatial.get_end_location(raw_df)
raw_df = spatial.create_distance_between_consecutive_column(raw_df)
raw_df = spatial.create_distance_from_start_column(raw_df)
spatial.get_distance_travelled_by_date_and_traj_id(raw_df, '2008-03-31', '10')
raw_df = spatial.create_point_within_range_column(raw_df, (40, 80), 100000)
raw_df = spatial.create_distance_from_given_point_column(raw_df, (40, 80))
raw_df = spatial.create_speed_from_prev_column(raw_df)
raw_df = spatial.create_acceleration_from_prev_column(raw_df)
raw_df = spatial.create_jerk_from_prev_column(raw_df)
raw_df = spatial.create_bearing_column(raw_df)
raw_df = spatial.create_bearing_rate_column(raw_df)
raw_df = spatial.create_rate_of_bearing_rate_column(raw_df)
spatial.get_distance_travelled_by_traj_id(raw_df, '10')
print(f"The total number of lines in the dataset are: {len(raw_df)}\n"
      f"The total number of trajectories in the dataset "
      f"are: {raw_df.reset_index()[const.TRAJECTORY_ID].nunique()}")

The total number of lines in the dataset are: 3387303
The total number of trajectories in the dataset are: 57
CPU times: user 44.2 s, sys: 8.38 s, total: 52.5 s
Wall time: 2min 3s
